# Modelowanie statystyczne 
> WSB-NLU, 2024  
> Andrzej Kocielski  
____

## Zaimportowanie potrzebnych modułów

In [68]:
import os

# do obliczń numerycznych i analizy statystycznej
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels as sm
import pingouin as pg

# do wizualizacji danych
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# zmiana wyświetlania liczb do trzech liczb po przecinku, bez notacji naukowej
# pd.set_option('display.float_format', lambda x: '%.3f' % x)

## Załadowanie danych surowych
Dane tygodniowe dla wybranych spółek oraz indeksów w okresie od 01.01.2023 do 15.12.2023. Źródło danych: [stooq.pl](https://stooq.pl/).

In [3]:
# Ścieżka do danych
directory_path = 'dane-finansowe'

# Lista plików csv
csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

print(f"Lista plików csv:")
for i, file in enumerate(csv_files):
    print(f"{i+1}. {file}") 

Lista plików csv:
1. comarch_w.csv
2. kghm_w.csv
3. eurpln_w.csv
4. spx_w.csv
5. echo_w.csv
6. xaupln_w.csv
7. apator_w.csv
8. wig_w.csv


In [4]:
# Przygotowanie danych roboczych

# Inicjalizaja pustego DataFrame
merged_df = pd.DataFrame()

# Zaczytanie danych z poszczególnych plików csv i dodanie do wspólnego DataFrame
for file in csv_files:
    file_path = os.path.join(directory_path, file)
    df = pd.read_csv(file_path)

    # Dodanie nazwy poliku do nazwy kolumny (poza pierwszą, czyli datą)
    nazwa_pliku = os.path.splitext(os.path.basename(file))[0][:-2].upper()
    df.columns = [df.columns[0]] + [f"{col}_{nazwa_pliku}" for col in df.columns[1:]]
    
    # Łączenie danych z poszczególnych plików 
    merged_df = pd.concat([merged_df, df], ignore_index=False, axis=1)

# Usunięcie powtórzonych kolumn z datami
merged_df = merged_df.T.drop_duplicates().T


In [5]:
# Zmiana typu danych 
cols = list(merged_df.columns)
cols.remove('Data')
for col in cols:
    merged_df[col] = merged_df[col].astype(float)

merged_df['Data'] = pd.to_datetime(merged_df['Data'])

In [6]:
print(f"Liczba obserwacji (wierszy): {merged_df.shape[0]}, liczba kolumn: {merged_df.shape[1]}")

Liczba obserwacji (wierszy): 1094, liczba kolumn: 39


In [7]:
print(f"Podgląd kilku pierwszych wierszy:")
merged_df.head()

Podgląd kilku pierwszych wierszy:


,Data,Otwarcie_COMARCH,Najwyzszy_COMARCH,Najnizszy_COMARCH,Zamkniecie_COMARCH,Wolumen_COMARCH,Otwarcie_KGHM,Najwyzszy_KGHM,Najnizszy_KGHM,Zamkniecie_KGHM,...,Otwarcie_APATOR,Najwyzszy_APATOR,Najnizszy_APATOR,Zamkniecie_APATOR,Wolumen_APATOR,Otwarcie_WIG,Najwyzszy_WIG,Najnizszy_WIG,Zamkniecie_WIG,Wolumen_WIG
0,2003-01-05,28.299,28.601,25.000,27.100,431401.754,13.653,14.045,13.301,13.896,...,18.011,18.135,17.538,18.135,6271.654,14397.800,14795.000,14210.200,14770.500,22771651.000
1,2003-01-12,27.501,28.801,27.402,27.802,177222.357,14.004,14.302,13.653,14.099,...,18.011,18.967,17.895,18.851,16115.659,14843.800,14917.600,14569.300,14785.900,32651395.000
2,2003-01-19,27.999,30.999,26.999,27.402,209181.153,14.248,14.451,13.856,13.950,...,18.610,20.640,18.610,19.919,45891.730,14865.700,15168.000,14578.900,14637.500,31098246.000
3,2003-01-26,27.299,28.500,26.900,27.100,74449.212,14.004,14.545,13.856,13.856,...,19.091,20.283,19.091,20.159,11392.845,14594.100,14654.000,14130.400,14133.800,26306693.000
4,2003-02-02,26.500,27.299,25.701,27.299,99876.092,13.599,13.694,12.951,13.599,...,20.041,20.993,18.967,20.758,12638.752,13930.100,13984.900,13623.700,13844.800,24134712.000


In [ ]:
# sprawdzamy brakujące dane (ich liczbę) w poszczególnych kolumnach 
merged_df.isnull().sum() 

___
## Statystyka opisowa
### Dla spółki Comarch (jako przykład)

In [8]:
comarch_cols = ["Otwarcie_COMARCH", "Zamkniecie_COMARCH", "Najnizszy_COMARCH", "Najwyzszy_COMARCH", "Wolumen_COMARCH"]

In [111]:
# własna funkcja do obliczania inter quartile range (IQR)
def q25(column):
    return column.quantile(0.25)
def q75(column):
    return column.quantile(0.75)    
def IQR(column): 
    q25, q75 = column.quantile([0.25, 0.75])
    return q75-q25

# własna funkcja do obliczania zakresu
def range_max_min(column):
    return column.max() - column.min()

# własna funkcja do obliczania współczynnika zmienności (coefficient of variance)
def cv(column):
    return stats.variation(list(column)) * 100

# własna funkcja do wyznaczania błędu standardowego
def sem(column):
    return stats.sem(list(column))

# własna funkcja do wyznaczenia przedziału ufności dla średniej
def ci(column):
    conf_level = 0.95
    df = len(column) - 1
    sem = stats.sem(list(column))
    return stats.t.interval(conf_level, df, np.mean(column), sem)

charakterystyka = ["min", q25, "mean", "median", q75, IQR, "max", range_max_min, "var", "std", cv, sem, ci, "skew", "kurtosis"]

In [112]:
merged_df[comarch_cols].agg(charakterystyka)

,Otwarcie_COMARCH,Zamkniecie_COMARCH,Najnizszy_COMARCH,Najwyzszy_COMARCH,Wolumen_COMARCH
min,26.500,26.999,25.000,27.299,414.209
q25,70.000,70.000,68.000,71.963,8167.880
mean,124.925,124.790,120.492,128.679,31502.620
median,116.959,117.211,112.854,119.996,16970.809
q75,176.000,176.000,169.148,179.996,36870.602
IQR,106.000,106.000,101.147,108.033,28702.722
max,275.000,273.000,260.000,280.000,431401.754
range_max_min,248.500,246.001,235.000,252.701,430987.545
var,3521.852,3496.817,3266.162,3724.534,1853908016.175
std,59.345,59.134,57.150,61.029,43057.032


### Interpretacja wybranych statystyk, np. dla _cen zamknięcia_ akcji Comarch w interwałach tygodniowych

$min$ -> najniższa cena akcji Comarch na zamknięciu tygodniowym w obserwowanym okresie

$max$ -> najwyższa cena akcji na zamknięciu tygodniowym 

$mean$ -> średnia arytmetyczna cen zamknięcia

$median$ -> mediana (wartość środkowa); wartość mniejsza od średniej wskazuje na skośność statystyki

$std$ -> odchylenie standardowe wyrażone w PLN; stosunkowo duża wartość $std$ w stosunku do średniej wskazuje na znaczny rozrzut zmiennej 

$cv$ -> współczynnik zmienności; średnia / odchylenie standardowe

$sem$ -> błąd standardowy średniej; rozrzut estymatorów z próby wokół parametru populacji, czyli jak bardzo średnia próbki jest "rozmyta" (miara niepewności testu)

$ci$ -> przedział ufności; z prawdopodobieństwem 0.95 prawdziwa wartość parametru populacyjnego (średnia cen zamknięcia) znajduje się w tym przedziale

$skew$ -> skośność; wartość większa od 1 świadczy o skośności lewostronnej

$kurtosis$ -> kurtoza; bada czy rozkład jest płaski, czy stromy; kurtoza < 0 wskazuje na rozkład platykurtyczny, czyli rozkład niski i szeroki (Wartości zmiennej są bardziej rozrzucone wokół średniej oraz mamy mniejsze prawdopodobieństwo wystąpienia wartości ekstremalnych)


## Interpretacja graficzna zbioru danych

In [ ]:
# przygotowanie podzbioru - tylko zamknięcia sesji
zamkniecia = ['Data', 'Zamkniecie_COMARCH', 'Zamkniecie_KGHM', 'Zamkniecie_EURPLN', 'Zamkniecie_SPX', 'Zamkniecie_ECHO', 'Zamkniecie_XAUPLN', 'Zamkniecie_APATOR', 'Zamkniecie_WIG']
zamkniecia_df = merged_df[zamkniecia]

spolki = ['Data', 'Zamkniecie_COMARCH', 'Zamkniecie_KGHM', 'Zamkniecie_ECHO', 'Zamkniecie_APATOR']
spolki_df = merged_df[spolki]

# ustawienie daty jako indeksu
zamkniecia_df.set_index('Data', inplace=True)
spolki_df.set_index('Data', inplace=True)

In [ ]:
# ustawienie stylu wykresów
# print(plt.style.available)
plt.style.use('classic')

In [ ]:
# wartość akcji w badanym okrecie
walor = zamkniecia_df.columns[-1]

plt.plot(zamkniecia_df[walor]) 
plt.xlabel('Czas')
plt.ylabel('Cena zamknięcia (PLN)')
plt.title(f'{walor}')
plt.xticks(rotation=35)  
plt.grid(True, linestyle='dotted',) 
plt.show()

In [ ]:
# wartość danego waloru w badanym okrecie
for walor in range(len(spolki_df.columns)):
    plt.plot(zamkniecia_df[spolki_df.columns[walor]], label=spolki_df.columns[walor]) 

plt.xlabel('Czas')
plt.ylabel('Cena zamknięcia (PLN)')
plt.title(f'Zamkniecia wybranych spółek')
plt.xticks(rotation=35)  
plt.grid(True,linestyle='dotted') 
plt.legend()
plt.show()

In [ ]:
# Create the first plot
fig, ax1 = plt.subplots()

# Plot the first dataset on the lebt y-axis
for walor in range(len(spolki_df.columns)):
    ax1.plot(zamkniecia_df[spolki_df.columns[walor]], linestyle='-', label=spolki_df.columns[walor]) 
ax1.set_xlabel('Czas')
ax1.set_ylabel('Cena zamknięcia (PLN)', color='grey')
ax1.tick_params('y', colors='grey')

# Create the second plot sharing the same x-axis
ax2 = ax1.twinx()

# Plot the second dataset on the right y-axis
wig = zamkniecia_df.columns[-1]
ax2.plot(zamkniecia_df[wig], color='black', linestyle='--', label=wig)
ax2.set_ylabel('Punkty indeksu', color='black')
ax2.tick_params('y', colors='black')

# Show the plots
lines, labels = ax1.get_legend_handles_labels()
lines_wig, labels_wig = ax2.get_legend_handles_labels()
ax2.legend(lines + lines_wig, labels + labels_wig, fontsize='10', loc='upper center')
plt.grid(True, linestyle='dotted') 
plt.show()

### Histogram dla przykładu cen EUR wyrażonej w PLN

In [ ]:
# Utwórzenie histogram
sns.histplot(merged_df['Zamkniecie_EURPLN'], bins=20, kde=False, color='skyblue')

# Opis wykresu i osi
plt.xlabel('EUR-PLN')
plt.ylabel('Częstość')
plt.title('Histogram - Wolumen WIG')

plt.show()

## Testy statystyczne
### Czy częstość występowania cen EUR-PLN ma rozkład normalny

In [122]:
# Test Kołmogorowa-Smirnowa
statistic, p_value = stats.kstest(merged_df['Zamkniecie_EURPLN'], 'norm')

print("Statystyka testowa:", statistic)
print("Wartość p:", p_value)

# Interpretacja wyników
alpha = 0.05
if p_value > alpha:
    print("Nie ma podstaw do odrzucenia hipotezy zerowej (o normalności rozkładu).")
else:
    print("Odrzucono hipotezę zerową na korzyść hipotezy alternatywnej (nie normalny rozklad).")

Statystyka testowa: 0.9993218667162591 , Wartość p: 0.0
Odrzucono hipotezę zerową na korzyść hipotezy alternatywnej (nie normalny rozklad).


In [123]:
# Test D'Agostino i Pearson:
statistic, p_value = stats.normaltest(merged_df['Zamkniecie_EURPLN'])

print("Statystyka testowa:", statistic)
print("Wartość p:", p_value)

# Interpretacja wyników
alpha = 0.05
if p_value > alpha:
    print("Nie ma podstaw do odrzucenia hipotezy zerowej (rozklad normalny).")
else:
    print("Odrzucono hipotezę zerową na korzyść hipotezy alternatywnej (nie normalny rozklad).")

Statystyka testowa: 60.17132120784457 , Wartość p: 8.589415573446294e-14
Odrzucono hipotezę zerową na korzyść hipotezy alternatywnej (nie normalny rozklad).


## Model regresji
### Na przykładzie cen zamknięcia spółki Comarch wobec indeksu WIG

## Dodatkowe źródła
- [https://mateuszgrzyb.pl/3-metody-analizy-normalnosci-rozkladu-w-python/](https://mateuszgrzyb.pl/3-metody-analizy-normalnosci-rozkladu-w-python/)
- [https://github.com/bilalonur/financial-visualization/blob/main/finance-visualization.ipynb](https://github.com/bilalonur/financial-visualization/blob/main/finance-visualization.ipynb)
- [https://python.cogsci.nl/numerical/statistics/](https://python.cogsci.nl/numerical/statistics/)
- [https://pingouin-stats.org/](https://pingouin-stats.org/)
- [https://www.naukowiec.org/wiedza/statystyka/](https://www.naukowiec.org/wiedza/statystyka/)

___
Andrzej Kocielski, 2024